### Import Lib

In [2]:
import pandas as pd
import os
import psycopg2
import numpy as np
import dask.dataframe as dd
import random
results = []
# import sqlalchemy as sql

### Create Connection to PostgreSQL

In [3]:
# Create Connection to postgresql

conn = psycopg2.connect(
    host="localhost",
    database="musicbrainz_db",
    user="musicbrainz",
    port=5432)

cursor = conn.cursor()

# psql -h localhost -p 25432 -U musicbrainz musicbrainz_db

# engine = sql.create_engine('postgresql:///musicbrainz_db')
# connection = engine.connect()
cursor.execute("select * from recording limit 1;")
cursor.fetchone()[0]
print("connection successful")

connection successful


### Select Random file paths

In [4]:
# #Load up MLHD Data Folder

# filelist = os.listdir('/data/mlhd/00')

# path = lambda name: '/data/mlhd/00/{}'.format(name)
# file_paths = [path(name) for name in filelist]

# first_10 = file_paths[:10]

In [5]:
# #random folder selector
# master_folder = os.listdir('/data/mlhd')
# folder_list = random.sample(master_folder, 10)

# #random file selection
# file_paths = []
# for folder in folder_list:
#     random_file = random.sample(os.listdir('/data/mlhd/'+(folder)), 1)[0]
# #     print(random_file)
#     path = '/data/mlhd/'+folder+'/'+random_file
#     file_paths.append(path)

# file_paths

### Warning: Running this code-block will replace existing random file_list

In [6]:
# with open('random_file_paths.txt', 'w+') as f:
#     f.writelines([name+'\n' for name in file_paths])

### Read frozen stored random files

In [7]:
with open('random_file_paths.txt', 'r') as f:
    file_paths = f.readlines()
    file_paths= [item.strip() for item in file_paths]
    
(file_paths)

['/data/mlhd/07/07beee31-451d-4346-9f83-144671d68f7b.txt.gz',
 '/data/mlhd/86/86482586-681c-43f6-8fda-05d0e5c3a5c3.txt.gz',
 '/data/mlhd/aa/aaaec8b6-6ec5-4efa-b9ee-ee089431d9c3.txt.gz',
 '/data/mlhd/0d/0d309017-bc04-4c2b-be84-ee01899fe087.txt.gz',
 '/data/mlhd/b3/b3ded736-58be-45ed-a018-5803c9eae282.txt.gz',
 '/data/mlhd/82/822abba8-6493-49ae-b70f-35f1ad8ca052.txt.gz',
 '/data/mlhd/56/567d224e-0660-4eaf-a6f5-34f7abe7e609.txt.gz',
 '/data/mlhd/4d/4d68890f-29d1-402d-8650-43941e1a3559.txt.gz',
 '/data/mlhd/ad/ad2ff9c6-7c80-4835-80b4-c117de9a2c2b.txt.gz',
 '/data/mlhd/e7/e7305d9a-260c-47b0-ba1b-01892300fa68.txt.gz']

In [8]:
# fetch df

# Use dask.dataframe since it's ~ 14x faster than pandas dataframe! 

def fetch_df_dd(pth):
#     df = dd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'], blocksize=None)
    df = pd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
    
    # DROPPING ALL FILES WITHOUT recording-MBID
    df = df[-df['recording-MBID'].isna()]
    return df

df = fetch_df_dd(file_paths[9])
print(len(df))
df.head()

10694


,timestamp,artist-MBID,release-MBID,recording-MBID
0,1238828757,71f17703-cd11-4579-b08a-1b1c8e6f5ac2,15efe0ab-055a-4562-b290-be39336e49bd,986dd819-7c83-4c9e-905b-6cb65ccbea3b
1,1238828982,71f17703-cd11-4579-b08a-1b1c8e6f5ac2,15efe0ab-055a-4562-b290-be39336e49bd,52d46c9b-96f4-4e59-8216-68f8b5ee2b89
2,1238829203,71f17703-cd11-4579-b08a-1b1c8e6f5ac2,15efe0ab-055a-4562-b290-be39336e49bd,200c74aa-7d6d-4a9b-8284-35094186a00b
4,1239496830,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,NaN,06bf6d0c-3094-492e-92fc-4867b9364d0b
5,1239497092,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,NaN,571b7e68-b7cf-4d33-97e2-b59eb7d00b9d


In [9]:
# A function to take single mbid as input and: 
# 1. cross check with recording table. If present, return the SAME recording-mbid from recording table.
# 2. If not in recording table, check in track table. If present, return 
# 3. If not in recording/track table, check if it exists in the recording_gid_redirect table
# 4. If not in any of the above tables, add the mbid to bad_entries list.

# Check if a recording-MBID exists in the "recording" table. If exists, return the mbid. If not, return None
def check_in_recording(rec_mbid):
    cursor.execute("SELECT gid FROM recording WHERE gid=%(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        return fetch[0]
    else:
        count_not_recording.add(rec_mbid)
        return None


# If MBID in recording_gid_redirect, return the redirected MBID. Else, return None
def check_in_redirect(rec_mbid):
    cursor.execute("select r.gid from recording r join recording_gid_redirect rgr on rgr.new_id = r.id where rgr.gid = %(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        return fetch[0]
    else:
        count_not_redirect.add(rec_mbid)
        return None


# Check if a recording-MBID exists in the "track" table. If not, return None. 
# If yes, append the track-mbid to the list track_mbids

track_mbids = []
def check_in_track(rec_mbid):
    cursor.execute("SELECT gid FROM track WHERE gid=%(id)s;", {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        track_mbids.add(rec_mbid)
        return fetch[0]
    else:
        count_not_track.add(rec_mbid)
        return None

# Simple function to take single mbid as input and:
# Find its canonical MBID
def get_canon(rec_mbid):
    cursor.execute("SELECT canonical_recording_mbid FROM mapping.canonical_recording_redirect WHERE recording_mbid=%(id)s;", {'id':rec_mbid})
    if cursor.fetchone()!=None:
        count_not_canonical.add(rec_mbid)
        return cursor.fetchone()
    else:
        return rec_mbid
    
# Function to validate MBIDs and replace MBIDS through a chain of functions.
def check_validity(rec_mbid):
    x = check_in_recording(rec_mbid)
    
    if x != None:
        return x
    else:
        x = check_in_redirect(rec_mbid)
    
        if x != None:
            return x
        else:            
            x = check_in_track(rec_mbid)
            
            if x!= None:
                return x
            else:
                return None

In [ ]:
# # query = "SELECT gid FROM track WHERE gid=%(id)s;"
# # cursor.execute(query,{'id':'625826fb-1380-4a0a-8b2a-03690e043c25'})


# tup = ('0f42ab32-22cd-4dcf-927b-a8d9a183d68b','327c8bf3-809b-4368-b590-7867a6cc805a')

# print(str(tup))
# query = '''SELECT id FROM recording 
# WHERE gid in ('0f42ab32-22cd-4dcf-927b-a8d9a183d68b','327c8bf3-809b-4368-b590-7867a6cc805a');'''
# cursor.execute(query)
# print(cursor.fetchall())

In [ ]:
rec = df['recording-MBID']
print(rec)
# print(check_in_recording(rec))
# # cursor.execute("select * from recording limit 1;")

0        986dd819-7c83-4c9e-905b-6cb65ccbea3b
1        52d46c9b-96f4-4e59-8216-68f8b5ee2b89
2        200c74aa-7d6d-4a9b-8284-35094186a00b
4        06bf6d0c-3094-492e-92fc-4867b9364d0b
5        571b7e68-b7cf-4d33-97e2-b59eb7d00b9d
                         ...                 
13354    4c3b6406-e144-434f-baab-372577764dbe
13355    5f27596a-3bd0-45b0-bd7e-d75ffc2674b0
13356    06fe79c6-f43f-46fd-acea-c3df21bb07a6
13357    fc8df6e8-8184-47b7-a9db-a36d53d0ea21
13358    c7bb57ec-3559-49e7-82a4-071f895b6190
Name: recording-MBID, Length: 10694, dtype: object


In [ ]:
# Declaring count sets
count_not_recording, count_not_redirect, count_not_track, count_misc, count_not_canonical = set(), set(), set(), set(), set()

# Driver function to check validity and get canononical MBID.
def driver_function(rec_mbid):
    x = check_validity(rec_mbid)
    # check_validity checks MBIDs in the following order: is recording-MBID? -> is redirectable-MBID? -> is track-MBID?
    # If its not either of the above 3, None is returned.
    
    if x != None:
        return get_canon(rec_mbid)
    # If its not None, find its canonical-MBID
    else:
        return None

In [ ]:
%%time
# df2 = df['recording-MBID'].map(driver_function, meta=('x', str)).compute()
df2 = df['recording-MBID'].map(driver_function)
# track_mbids

CPU times: user 865 ms, sys: 981 ms, total: 1.85 s
Wall time: 19.6 s


# Artist Conflation Analysis

### Problem Statement:
Retrieve a chunk of recording-MBIDs, get their respective artist-MBIDs, and compare these newly found artist-MBIDs with the ones already in MLHD.

### Solution:
1. Retrieve a chunk of artist & recording-MBIDs
2. write a function to accept a series of recording-MBIDs, and get their respective artist-MBIDs
3. Use the above function to get artist-MBIDs. Concat these new-MBIDs with the original table
4. Now compare new artist-MBIDs with the existing ones. list the recording-MBIDs that don't match.
5. Publish results and get them checked with alastair. 

In [ ]:
# Function to yield a chunk from dataframe.

def yield_chunk(dataframe, chunksize):
    l = len(dataframe)
    if l<chunksize:
        return dataframe
    
    for i in range(chunksize,l,chunksize):
        start = i-chunksize
        yield dataframe[start:i].loc[:, ('recording-MBID', 'artist-MBID')]
        
gen = yield_chunk(df, 5)
next(gen)

In [2]:
# Write a function to accept 
# select artist_mbids from mapping.canonical_musicbrainz_data where recording_mbid in ();

def check_in_redirect(rec_mbid):
    query = "select artist_mbids from mapping.canonical_musicbrainz_data where recording_mbid in %(id)s;"
    cursor.execute(query, {'id':rec_mbid})
    fetch = cursor.fetchone()
    
    if type(fetch) == tuple:
        return fetch[0]
    else:
        count_not_redirect.add(rec_mbid)
        return None